**Kaggle Competition**

https://www.kaggle.com/c/plant-seedlings-classification

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/My Drive/')

In [ ]:
from zipfile import ZipFile

In [ ]:
with ZipFile('/content/drive/My Drive/test.zip', 'r') as z:
  z.extractall()

In [ ]:
with ZipFile('/content/drive/My Drive/train.zip', 'r') as z:
  z.extractall()

In [ ]:
x_test=[]
os.chdir('/content/drive/My Drive/test')
import cv2
for i in os.listdir():
    dummy = cv2.imread(i)
    dummy = cv2.resize(dummy,(128,128))
    x_test.append(dummy)

In [ ]:
x_train = []
y_train = []

In [ ]:
os.chdir('/content/drive/My Drive/train')

In [ ]:
os.listdir()

['Fat Hen',
 'Small-flowered Cranesbill',
 'Cleavers',
 'Black-grass',
 'Sugar beet',
 'Shepherds Purse',
 'Charlock',
 'Loose Silky-bent',
 'Scentless Mayweed',
 'Maize',
 'Common Chickweed',
 'Common wheat']

In [ ]:
x_train = []
y_train = []
import cv2
for i in os.listdir():
    print(i)
    if (os.path.isdir(i)):
            for j in os.listdir(i):
                try:
                    dummy = cv2.imread('/content/drive/My Drive/train/' + i + "/" + j)
                    dummy = cv2.resize(dummy,(128,128))
                    x_train.append(dummy)
                    y_train.append(i)
                except Exception as e:
                    print(e)


Fat Hen
Small-flowered Cranesbill
Cleavers
Black-grass
Sugar beet
Shepherds Purse
Charlock
Loose Silky-bent
Scentless Mayweed
Maize
Common Chickweed
Common wheat


In [ ]:
x_train[0].shape

(128, 128, 3)

In [ ]:
import pandas as pd
dum = pd.get_dummies(y_train)

In [ ]:
encoded_labels = dum
y_train = dum

In [ ]:
import numpy as np
y_train = np.array(y_train)

In [ ]:
x_train = np.array(x_train)

In [ ]:
x_train[0].shape

(128, 128, 3)

In [ ]:
from sklearn.model_selection import train_test_split
x_train2, x_val, y_train2, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2)
print (len(x_train2))
print (len(x_val))

3800
950


In [ ]:
x_train2[0].shape

(128, 128, 3)

In [ ]:
x_train2 = x_train2.reshape(x_train2.shape[0],128,128,3)

In [ ]:
x_val = x_val.reshape(x_val.shape[0],128,128,3)

In [ ]:
x_train2 = x_train2/255.

x_val = x_val/255.

In [ ]:
print (x_train2.shape)
print (x_val.shape)

(3800, 128, 128, 3)
(950, 128, 128, 3)


In [ ]:
print(y_train2.shape)
print(y_val.shape)

(3800, 12)
(950, 12)


In [ ]:
from keras import applications
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (128, 128, 3))

In [ ]:
# Freeze the layers which you don't want to train. In this we are freezing the first 5 layers.
for layer in model.layers[:5]:
    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(64, activation="relu")(x)
x = Dense(32, activation="relu")(x)
predictions = Dense(12, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001), metrics=["accuracy"])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [ ]:
# Save the model 
checkpoint = ModelCheckpoint("vgg16_best.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')

epochs=20
# Train the model 
model_final.fit(x_train2, y_train2, epochs = epochs, validation_data=(x_val, y_val), callbacks = [checkpoint, early])

Train on 3800 samples, validate on 950 samples
Epoch 1/20
3800/3800 [==============================] - 40s 11ms/step - loss: 2.3824 - acc: 0.1603 - val_loss: 2.1161 - val_acc: 0.2895

Epoch 00001: val_acc improved from -inf to 0.28947, saving model to vgg16_best.h5
Epoch 2/20
3800/3800 [==============================] - 38s 10ms/step - loss: 1.7014 - acc: 0.4161 - val_loss: 1.3077 - val_acc: 0.5211

Epoch 00002: val_acc improved from 0.28947 to 0.52105, saving model to vgg16_best.h5
Epoch 3/20
3800/3800 [==============================] - 38s 10ms/step - loss: 1.0564 - acc: 0.6226 - val_loss: 1.0295 - val_acc: 0.6400

Epoch 00003: val_acc improved from 0.52105 to 0.64000, saving model to vgg16_best.h5
Epoch 4/20
3800/3800 [==============================] - 38s 10ms/step - loss: 0.7512 - acc: 0.7432 - val_loss: 0.8915 - val_acc: 0.6653

Epoch 00004: val_acc improved from 0.64000 to 0.66526, saving model to vgg16_best.h5
Epoch 5/20
3800/3800 [==============================] - 38s 10ms/ste